In [1]:
#### this notebook was created to properly implement NCF mode
#### this code cannor pick target items for random selection

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.utils import shuffle

import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras import layers,Input, Model
from tensorflow.keras.layers import Dense, Concatenate, Multiply, Embedding,Dot, Dropout
from tensorflow.keras.layers import *

In [3]:
LATENT_DIM = 16 ### basically embedding_dimension
DROP_RATE = 0.9 # dropout rate for dropout layer 
TRAIN_NEG_NUM = 4 # negative examples for train set
TEST_NEG_NUM = 100 # negative examples for test set
NUM_EPOCHS = 10
BATCH_SIZE = 256

In [4]:
u_item_link = 'https://raw.githubusercontent.com/malinphy/datasets/main/ml100K/u_item.csv'
item_df = pd.read_csv(u_item_link)
item_df = item_df[['movie id', 'movie title']]
item_df = item_df.rename(columns={"movie id": "item_id","movie title": "movie_title"})
# print('unique_item_id',len(item_df['item_id'].unique()))
# print('unique_movie_title',len(item_df['movie_title'].unique()))

# dup_items = np.where(item_df.duplicated(subset = ['movie_title']) == True)[0]
# # print(dup_items)
# titles = []

# for i in dup_items:
#     x = str(item_df.iloc[dup_items]['movie_title'][i])
#     titles.append(x)

# for i in titles:
#     print(item_df[item_df['movie_title'] == i])
# """
# as seen from number of unique item and number of unique movie title
# are different from each other. Therefore, it is concluded that 
# some data points are duplicated. I prefer to drop one of the duplicated item .
# """
# item_df = item_df.drop_duplicates(subset = ['movie_title'],keep ='last').reset_index(drop = True)

In [5]:
def encoder_2(df, col):
    df = df.sort_values(by = [col]).reset_index(drop = True)
    unique_features = df[col].unique()
    feature_2enc = {i:j for i,j in enumerate(df[col].unique())}
    enc_2feature = {j:i for i,j in enumerate(df[col].unique())}

    return feature_2enc, enc_2feature

def sequencer(df,col):
    var = df.groupby(col).aggregate(lambda tdf: tdf.unique().tolist()) 
    var = var.reset_index()

    return var

def train_test_maker(df,col): ### df will be in sequential format
    last_items = []
    for i in range(len(df)):
        last_items.append(np.array(df[col][i][-1]))

    mid_items = []
    for i in range(len(df)):
        mid_items.append(df[col][i][:-1])

    return (last_items, mid_items)

def train_test_maker(df,col): ### df will be in sequential format
    last_items = []
    for i in range(len(df)):
        last_items.append(np.array(df[col][i][-1]))

    mid_items = []
    for i in range(len(df)):
        mid_items.append(df[col][i][:-1])

    return (last_items, mid_items)    

def train_neg_maker(num_neg, df,col):
    num_neg = num_neg
    neg3 = []
    ui= 0
    us = []
    for i in df[col]:
        pos_set = i
        neg2 = []
  
        for j in range(num_neg):
            neg1 = []
            for k in pos_set:
      
                neg_candidate = np.random.randint(1,len(unique_item_enc))
                while neg_candidate in pos_set:
                    neg_candidate = np.random.randint(1,len(unique_item_enc))
                us.append(ui)
      
                neg1.append((neg_candidate))
            neg2.append(np.array(neg1))
        ui += 1
        neg3.append(neg2)
    
    return us, neg3

def test_negative_maker(num_neg,df,col):
    num_neg = num_neg
    neg3 = []
    ui= 0
    us = []
    neg2 = []
    for i in df[col]:
        pos_set = i
  
        neg1 = []
        for j in range(num_neg):
    
            neg_candidate = np.random.randint(1,len(unique_item_enc))

            while neg_candidate == pos_set:

                neg_candidate = np.random.randint(1,len(unique_item_enc))
            us.append(ui)
      
            neg1.append((neg_candidate))
        neg2.append(np.array(neg1))

    return neg2

In [6]:
item_2title = { item_df['item_id'][i] : item_df['movie_title'][i]  for i in range(len(item_df))}
title_2item = { item_df['movie_title'][i] : item_df['item_id'][i]   for i in range(len(item_df))}


In [7]:
u_data_link = 'https://raw.githubusercontent.com/malinphy/datasets/main/ml100K/u.data'
df = pd.read_csv(u_data_link,delimiter='\t',header=None)
col_titles = ['user_id', 'item_id' , 'rating' , 'timestamp']
df.columns = col_titles
df = df.drop(columns = ['timestamp'])

num_users = len(df['user_id'].unique())
num_items = len(df['item_id'].unique())

print('number of different users :', num_users)
print('number of different items :', num_items)

number of different users : 943
number of different items : 1682


In [8]:
user_2enc, enc_2user = encoder_2(df, 'user_id')
item_2enc, enc_2item = encoder_2(df, 'item_id')

In [9]:
# user_2enc.keys()
# user_2enc.values()
df['user_id_enc']= df['user_id'].map(enc_2user).astype('int32')
df['item_id_enc']= df['item_id'].map(enc_2item).astype('int32')
unique_user_enc = user_2enc.keys()
unique_item_enc = item_2enc.keys()

df_enc = df[['user_id_enc','item_id_enc','rating']]
df_enc.head(3)

,user_id_enc,item_id_enc,rating
0,195,241,3
1,185,301,3
2,21,376,1


In [10]:
df_2 = df_enc[['user_id_enc','item_id_enc','rating']]
df_2['rating'] = 1
df_enc_seq =sequencer(df_2,'user_id_enc')
df_enc_seq.head(3)

,user_id_enc,item_id_enc,rating
0,0,"[60, 188, 32, 159, 19, 201, 170, 264, 154, 116...",[1]
1,1,"[291, 250, 49, 313, 296, 289, 311, 280, 12, 27...",[1]
2,2,"[334, 244, 336, 342, 322, 330, 293, 331, 327, ...",[1]


In [11]:
last_items, mid_items = train_test_maker(df_enc_seq,'item_id_enc')

test_df_seq = pd.DataFrame({'user_id_enc':unique_user_enc,
                        'item_id_enc':np.array(last_items),
                        'rating':np.ones(len(last_items),dtype = 'int32')})

train_df_seq = pd.DataFrame({'user_id_enc':unique_user_enc,
                        'item_id_enc':mid_items,
                        'rating':np.ones(len(mid_items),dtype = 'int32')})

test_df = pd.DataFrame({'user_id_enc':df_enc_seq['user_id_enc'].copy(),
                        'item_id_enc':last_items, 
                        'rating' : np.ones(len(last_items),dtype='int32') 
                         })


In [12]:
last_items, mid_items = train_test_maker(df_enc_seq,'item_id_enc')

test_df_seq = pd.DataFrame({'user_id_enc':unique_user_enc,
                        'item_id_enc':np.array(last_items),
                        'rating':np.ones(len(last_items),dtype = 'int32')})

train_df_seq = pd.DataFrame({'user_id_enc':unique_user_enc,
                        'item_id_enc':mid_items,
                        'rating':np.ones(len(mid_items),dtype = 'int32')})

test_df = pd.DataFrame({'user_id_enc':df_enc_seq['user_id_enc'].copy(),
                        'item_id_enc':last_items, 
                        'rating' : np.ones(len(last_items),dtype='int32') 
                         })

In [13]:
user_create = []
for i in range(len(mid_items)):
  
  user_create.append(np.full(len(mid_items[i]),i))

train_df = pd.DataFrame({'user_id_enc':np.concatenate(user_create),
                        'item_id_enc':np.concatenate(mid_items), 
                        'ratings' : np.ones(len(np.concatenate(user_create)),dtype='int32') 
                         })

In [14]:
us , neg3 = train_neg_maker(TRAIN_NEG_NUM,df_enc_seq,'item_id_enc')

In [15]:
train_neg = pd.DataFrame({'user_id_enc':us,
              'item_id_enc':np.concatenate(np.concatenate(np.array(neg3))),
              'ratings':np.zeros(len(us), dtype='int32')})
train_neg.head(3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


,user_id_enc,item_id_enc,ratings
0,0,737,0
1,0,488,0
2,0,924,0


In [16]:
test_negs = test_negative_maker(TEST_NEG_NUM,test_df_seq,'item_id_enc')
test_negatives = pd.DataFrame(test_negs)
test_negatives
test_negatives.columns = np.arange(1,TEST_NEG_NUM+1)

train_total = shuffle(pd.concat([train_neg,train_df]))

In [17]:
def model(num_users, num_items):
    emb_dim = 16

    user_input = Input(shape = (1,), name = 'user_input')
    item_input = Input(shape = (1,), name = 'item_input')

    user_emb_mlp = Embedding(num_users+1, LATENT_DIM, name = 'user_emb_mlp')(user_input)
    item_emb_mlp = Embedding(num_items+1, LATENT_DIM, name = 'item_emb_mlp')(item_input)

    user_emb_gmf = Embedding(num_users+1, LATENT_DIM, name = 'user_emb_gmf')(user_input)
    item_emb_gmf = Embedding(num_items+1, LATENT_DIM, name = 'item_emb_gmf')(item_input)

    mult_layer = Multiply(name = 'element_wise_multiplication_gmf'
                    )([user_emb_gmf, item_emb_gmf])

    concat_layer = Concatenate()([user_emb_mlp,item_emb_gmf])

    dense_1 = Dense(64, activation = 'relu', name = 'dense_1_mlp')(concat_layer)
    dense_1 = Dropout(DROP_RATE)(dense_1)
    dense_2 = Dense(32, activation = 'relu', name = 'dense_2_mlp')(dense_1)
    dense_2 = Dropout(DROP_RATE)(dense_2)
    dense_3 = Dense(16, activation = 'relu', name = 'dense_3_mlp')(dense_2)
    dense_3 = Dropout(DROP_RATE)(dense_3)
    dense_4 = Dense(8, activation = 'relu')(dense_3)
    dense_4 = Dropout(DROP_RATE)(dense_4)
    # dense_5 = Dense(4, activation = 'relu')(dense_4)

    neuMF_layer = Concatenate(axis=-1, name = 'NeuMF_layer')([mult_layer,dense_4])

    final = Dense(1, activation = 'sigmoid', name = 'final_layer')(neuMF_layer)

    return Model(inputs = [user_input, item_input], outputs = [final])
mlp_model = model(num_users, num_items)
mlp_model.compile(
    # loss = tf.keras.losses.BinaryCrossentropy(),
    loss = 'BinaryCrossentropy',
    optimizer = 'Adam',
    metrics = ['accuracy']
    )

mlp_hist = mlp_model.fit([train_total['user_id_enc'], train_total['item_id_enc']],
              train_total['ratings'],
              epochs = 20,
            #   verbose = 0,
            #   batch_size = BATCH_SIZE
              )

Epoch 1/20
15596/15596 [==============================] - 61s 4ms/step - loss: 0.3869 - accuracy: 0.8353
Epoch 2/20
15596/15596 [==============================] - 54s 3ms/step - loss: 0.2823 - accuracy: 0.8763
Epoch 3/20
15596/15596 [==============================] - 52s 3ms/step - loss: 0.2566 - accuracy: 0.8888
Epoch 4/20
15596/15596 [==============================] - 51s 3ms/step - loss: 0.2424 - accuracy: 0.8959
Epoch 5/20
15596/15596 [==============================] - 47s 3ms/step - loss: 0.2336 - accuracy: 0.8998
Epoch 6/20
15596/15596 [==============================] - 47s 3ms/step - loss: 0.2277 - accuracy: 0.9028
Epoch 7/20
15596/15596 [==============================] - 48s 3ms/step - loss: 0.2231 - accuracy: 0.9052
Epoch 8/20
15596/15596 [==============================] - 46s 3ms/step - loss: 0.2195 - accuracy: 0.9067
Epoch 9/20
15596/15596 [==============================] - 47s 3ms/step - loss: 0.2167 - accuracy: 0.9080
Epoch 10/20
15596/15596 [==============================

In [18]:
test_preds = []
for i in range(len(test_negs)):
    p_user = np.full(len(test_negs[0])+1,i, dtype='int32').reshape(-1, 1)  
    new_test = np.concatenate((np.array(test_negs[i]),np.array([test_df['item_id_enc'][i]])))
    # x = tf.math.top_k(tf.squeeze(tf.squeeze(mlp_model.predict([p_user,test_negs[i]]), axis = 1), axis =1),k =100)[1]
    x = tf.math.top_k(tf.squeeze(tf.squeeze(mlp_model.predict([p_user,new_test]), axis = 1), axis =1),k =10)[1]
    # y = test_negs[i]
    y = new_test
    test_preds.append(y[np.array(x)])
    # test_preds.append(tf.math.top_k(tf.squeeze(tf.squeeze(mlp_model.predict([p_user,test_negs[i]]), axis = 1), axis =1),k =10)[1])

###EVALUATION
for evaluation, hit rate @10 will be calculated

In [19]:
def hit_calculator(leftOut, top_preds):
    hits = []
    total = 0
    h= 0
    for i in range(len(leftOut)):
        # print(i)
        if leftOut[i] in top_preds[i]:
            hits.append(1)
            h +=1
        else :
            hits.append(0)
    total +=1
    return hits, h/total

total_hits,sonuc =hit_calculator(test_df['item_id_enc'],test_preds)
np.mean(total_hits)

0.6839872746553552

In [20]:
def infer_func(user_id):
    p_user = np.full((num_items-1), user_id, dtype='int32').reshape(-1, 1)  
    user_preds = mlp_model.predict([p_user, np.arange(1,num_items).reshape(-1,1)])
    user_preds = tf.math.top_k(tf.squeeze(tf.squeeze(user_preds, axis = 1), axis =1),k =10)[1]
    return user_preds

In [21]:
for i in list(np.array(infer_func(3))):
    print(item_2title[enc_2item[i]])

Ulee's Gold (1997)
My Best Friend's Wedding (1997)
Midnight in the Garden of Good and Evil (1997)
Crash (1996)
Dante's Peak (1997)
Conan the Barbarian (1981)
Desperate Measures (1998)
Full Monty, The (1997)
Nosferatu (Nosferatu, eine Symphonie des Grauens) (1922)
Ruling Class, The (1972)
